In [46]:
from requests_html import HTMLSession # for making request
import requests # for making request
import pandas as pd # for data processing
import numpy as np # for data processing
from tqdm import tqdm # for count time of iteration
import re
from tqdm import tqdm
from imdb import IMDb
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Problem

Prediction đóng một vai trò khá quan trọng trong nhiều **Recommendation Systems**.

Phần này sẽ là phần chính của đồ án, gồm 3 task prediction:
- Content Based Filtering: predict những bộ phim khác từ một bộ phim dựa trên những thuộc tính của nó như title, đạo diễn, diễn viên, thể loại, overview,... (dùng TF-IDF để vectorize, sau đó dùng cosine similarity matrix để dự đoán)
- Content Based Filtering: predict điểm số mà user có thể muốn cho một phim nào đó (dùng SVD)
- Collaborative Filtering: predict những bộ phim cho một user dựa trên những users có độ liên quan gần nhau (KNN)

Task predict đầu tiên sẽ phù hợp nếu chúng ta không biết user là ai.
2 task predict sau, bằng sự kết hợp lẫn nhau chọn ra top phim chung, sẽ phù hợp để recommend từ sở thích của user.

# Dự đoán phim dựa vào thuộc tính riêng

Ở bước này, ta thường thấy rằng một bộ phim có sự tương đồng với bộ phim khác nằm ở thể loại, kịch bản, cốt truyện, và thậm chí là diễn viên, đạo diễn.

Thế nên, mình sẽ lấy hết text từ các trường title, overview, genres từ file movie.csv, cùng toàn bộ các trường của file credit.csv, join hai file bằng movie_id

In [6]:
credit_df = pd.read_csv('../data/credit.csv', dtype={'id':str})
movie_df = pd.read_csv('../data/movie.csv', dtype={'id':str})

In [11]:
movie_credit_df = pd.merge(movie_df, credit_df, left_on='id', right_on='id', how='outer')

## Clean data bước 1:

Vì các field của credit đều có dạng list[dictionary] nên mình sẽ chuyển nó về dạng list[name], và lấy tối đa top 3 trong list đó

In [16]:
features = ['cast', 'directors', 'writers', 'producers', 'composers']
for feature in features:
    movie_credit_df[feature] = movie_credit_df[feature].apply(literal_eval)

In [20]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [19]:
features = ['cast', 'directors', 'writers', 'producers', 'composers']
for feature in features:
    movie_credit_df[feature] = movie_credit_df[feature].apply(get_list)

In [30]:
movie_credit_df[['cast', 'directors', 'writers', 'producers', 'composers', 'genres']].head(5)

,cast,directors,writers,producers,composers,genres
0,"[amitabhbachchan, hemamalini, salmankhan]",[ravichopra],"[shafiqansari, satishbhatnagar, b.r.chopra]","[ashwanichopra, b.r.chopra]","[aadeshshrivastava, uttamsingh]",drama;romance
1,"[donaldpleasence, paulrudd, mariannehagan]",[joechappelle],"[debrahill, johncarpenter, danielfarrands]","[malekakkad, moustaphaakkad, paulfreeman]","[alanhowarth, paulrabjohns]",horror;thriller
2,"[kevindepaula, leonardolimacarvalho, seujorge]","[jeffzimbalist, michaelzimbalist]","[jeffzimbalist, michaelzimbalist]","[alexandredauman, guyeast, caíquemartinsferreira]",[a.r.rahman],biography;drama;sport
3,"[giuseppebattiston, annafoglietta, marcogiallini]",[paologenovese],"[filippobologna, paolocostella, paologenovese]","[marcobelardi, ughettacurto, marcogiannoni]",[mauriziofilardo],comedy;drama
4,"[hilaryduff, oliverjames, davidkeith]",[seanmcnamara],"[mitchrotter, samschreiber]","[davidbrookwell, a.j.dix, tobyemmerich]","[machinehead, aaronzigman]",family;music;musical;romance


## Clean data bước 2:

Mình sẽ chuyển tên các diễn viên về thành chữ thường và xoá khoảng trắng: ví dụ Johny Deep thành johnydeep. Điều này cũng giúp tránh phải hiện tượng trùng tên khác họ, ví dụ chữ Johny trong 2 cái tên khác nhau sẽ là 2 người khác nhau, nhưng nó có thể dẫn đến sự giống nhau khi vectorize, do đó phải tìm cách để nó có sự khác biệt, và xoá dấu cách như trên nhằm giải quyết vấn đề này

In [25]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [28]:
features = ['cast', 'directors', 'writers', 'producers', 'composers', 'genres']
for feature in features:
    movie_credit_df[feature] = movie_credit_df[feature].apply(clean_data)

In [29]:
movie_credit_df[['cast', 'directors', 'writers', 'producers', 'composers', 'genres']].head(5)

,cast,directors,writers,producers,composers,genres
0,"[amitabhbachchan, hemamalini, salmankhan]",[ravichopra],"[shafiqansari, satishbhatnagar, b.r.chopra]","[ashwanichopra, b.r.chopra]","[aadeshshrivastava, uttamsingh]",drama;romance
1,"[donaldpleasence, paulrudd, mariannehagan]",[joechappelle],"[debrahill, johncarpenter, danielfarrands]","[malekakkad, moustaphaakkad, paulfreeman]","[alanhowarth, paulrabjohns]",horror;thriller
2,"[kevindepaula, leonardolimacarvalho, seujorge]","[jeffzimbalist, michaelzimbalist]","[jeffzimbalist, michaelzimbalist]","[alexandredauman, guyeast, caíquemartinsferreira]",[a.r.rahman],biography;drama;sport
3,"[giuseppebattiston, annafoglietta, marcogiallini]",[paologenovese],"[filippobologna, paolocostella, paologenovese]","[marcobelardi, ughettacurto, marcogiannoni]",[mauriziofilardo],comedy;drama
4,"[hilaryduff, oliverjames, davidkeith]",[seanmcnamara],"[mitchrotter, samschreiber]","[davidbrookwell, a.j.dix, tobyemmerich]","[machinehead, aaronzigman]",family;music;musical;romance


## Vectorize và predict function

In [40]:
def create_soup(x):
    ans = x['overview'] + ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['directors']) + ' ' + ' '.join(x['genres'].split(";")) + ' ' + ' '.join(x['writers']) + ' ' + ' '.join(x['producers']) + ' ' + ' '.join(x['composers'])
    return ans

In [41]:
movie_credit_df['soup'] = movie_credit_df.apply(create_soup, axis=1)

In [42]:
movie_credit_df.head(5)

,id,title,runtimes,genres,vote_counts,average_rating,overview,cast,directors,writers,producers,composers,soup
0,0337578,Gardener,181,drama;romance,15084,7.4,Raj Malhotra and wife Pooja have four sons. Th...,"[amitabhbachchan, hemamalini, salmankhan]",[ravichopra],"[shafiqansari, satishbhatnagar, b.r.chopra]","[ashwanichopra, b.r.chopra]","[aadeshshrivastava, uttamsingh]",Raj Malhotra and wife Pooja have four sons. Th...
1,0113253,Halloween: The Curse of Michael Myers,87,horror;thriller,31301,4.8,Six years after Michael Myers' last massacre i...,"[donaldpleasence, paulrudd, mariannehagan]",[joechappelle],"[debrahill, johncarpenter, danielfarrands]","[malekakkad, moustaphaakkad, paulfreeman]","[alanhowarth, paulrabjohns]",Six years after Michael Myers' last massacre i...
2,0995868,Pele: Birth of a Legend,107,biography;drama;sport,16402,7.2,Pele's meteoric rise from the slums of Sao Pau...,"[kevindepaula, leonardolimacarvalho, seujorge]","[jeffzimbalist, michaelzimbalist]","[jeffzimbalist, michaelzimbalist]","[alexandredauman, guyeast, caíquemartinsferreira]",[a.r.rahman],Pele's meteoric rise from the slums of Sao Pau...
3,4901306,Perfect Strangers,96,comedy;drama,56387,7.8,"On a warm summer evening, the loving couple of...","[giuseppebattiston, annafoglietta, marcogiallini]",[paologenovese],"[filippobologna, paolocostella, paologenovese]","[marcobelardi, ughettacurto, marcogiannoni]",[mauriziofilardo],"On a warm summer evening, the loving couple of..."
4,0361696,Raise Your Voice,107,family;music;musical;romance,26730,5.9,This film is about a teenage girl who is very ...,"[hilaryduff, oliverjames, davidkeith]",[seanmcnamara],"[mitchrotter, samschreiber]","[davidbrookwell, a.j.dix, tobyemmerich]","[machinehead, aaronzigman]",This film is about a teenage girl who is very ...


In [44]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movie_credit_df['soup'])

In [49]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [56]:
def get_recommendations(title, cosine_sim=cosine_sim, top=10):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:top+1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movie_credit_df['title'].iloc[movie_indices]

In [57]:
movie_credit_df = movie_credit_df.reset_index()
indices = pd.Series(movie_credit_df.index, index=movie_credit_df['title'])

## Thế là xong, mình sẽ thử recommend theo vài bộ phim

Có thể thấy, ở recommend đầu tiên, những bộ phim nói về chủ đề người dơi sẽ được recommend nhiều, cũng như các bộ phim thuộc về hành động sẽ vào top similar với bộ phim mình chọn.

Ở recommend thứ hai, rõ ràng những bộ phim nằm trong vũ trụ Marvel đều lọt top khá cao nhờ vào sự giống nhau không chỉ nội dung mà còn diễn viên, đạo diễn, ...

In [58]:
get_recommendations('The Dark Knight Rises', cosine_sim, top=20)

7725                          The Dark Knight
3373                            Batman Begins
3508                             Interstellar
8564                             Man of Steel
5110                            Crime Busters
5932                                   Batman
2538                             The Prestige
429                                   Memento
798                                   Dunkirk
5683                      Hobo with a Shotgun
3986                                Following
4568                           28 Weeks Later
1153       National Treasure: Book of Secrets
5992             Batman: Mask of the Phantasm
5083                    The Lego Batman Movie
1168                           Batman Forever
797     Sky Captain and the World of Tomorrow
933                                The Shadow
244               Resident Evil: Degeneration
8248                                      2.0
Name: title, dtype: object

In [59]:
get_recommendations('Avengers: Age of Ultron', cosine_sim, top = 20)

4090                     Avengers: Infinity War
2446                               The Avengers
2584                                 Iron Man 3
8001                                 Enemy Mine
6758                                 Iron Man 2
3679                      Rise of the Guardians
579                                    Geostorm
490                                    Spy Kids
631                           Avengers: Endgame
5421            The Divergent Series: Insurgent
7431    Spy Kids 4-D: All the Time in the World
2562                         Sonic the Hedgehog
8188                           Superman Returns
238                                    Iron Man
2294                         The Brothers Bloom
6743                                   Serenity
4451                                       2012
2675        Evangelion: 1.0 You Are (Not) Alone
1256                              Power Rangers
608                          Mad Max: Fury Road
Name: title, dtype: object